In [144]:
import numpy as np
from pynq import Overlay
import time

In [145]:
ol = Overlay('qrf.bit')

In [146]:
ol?

In [147]:
dma_main = ol.axi_dma_1
dma2 = ol.axi_dma_0

In [148]:
dma_main?

In [149]:
A_channel = dma_main.sendchannel
Q_channel = dma_main.recvchannel
R_channel = dma2.recvchannel

In [150]:
import random

In [151]:
np.random.seed(1)
data_A = np.random.uniform(low=1, high=2, size=(50, 50)).astype(np.float32)

In [152]:
data_A

array([[1.417022 , 1.7203245, 1.0001143, ..., 1.2936141, 1.2877754,
        1.1300286],
       [1.019367 , 1.6788355, 1.2116281, ..., 1.5736794, 1.0028703,
        1.617145 ],
       [1.3266449, 1.5270581, 1.8859421, ..., 1.712989 , 1.5597169,
        1.012556 ],
       ...,
       [1.1466087, 1.6124611, 1.2793204, ..., 1.0070467, 1.0595286,
        1.4814005],
       [1.9257041, 1.4055574, 1.0750326, ..., 1.1667219, 1.5208484,
        1.7402936],
       [1.4624745, 1.8696141, 1.660412 , ..., 1.5681397, 1.3434201,
        1.3506172]], dtype=float32)

In [153]:
np.shape(data_A)

(50, 50)

In [154]:
def givensrotation_complex(a, b):
    z = a*a + b*b
    r = np.sqrt(z.real**2 + z.imag**2)
    theta = np.arctan2(z.imag, z.real)
    hypo = np.sqrt(r) * (np.cos(theta / 2) + 1j * np.sin(theta / 2))
    if hypo == 0:
        return 1, 0
    else:
        cos1 = a/ hypo
        sin1 = -b / hypo
        return cos1, sin1

In [155]:
def givensrotation_R(a, b):
    if b == 0:
        return 1.0, 0.0
    r = np.hypot(a, b)
    c = a / r
    s = -b / r
    return c, s


In [156]:
def qr_givens(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m,dtype=np.float64)
    for i in range(0, n - 1):
        for j in range(i + 1, m):
            cos, sin = givensrotation_R(R[i, i], R[j, i])
            R[i], R[j] = (R[i]*cos) + (R[j]*(-sin)), (R[i]*sin) + (R[j] * cos)
            Q[i], Q[j] = (Q[i]*cos) + (Q[j]*(-sin)), (Q[i]*sin) + (Q[j] * cos)
    return np.transpose(Q), R

In [157]:
start = time.time()
q1,r1 = qr_givens(data_A)
fin = time.time()
pstime = fin-start
print(pstime)

0.2488996982574463


In [158]:
start = time.time()
q2,r2 = np.linalg.qr(data_A, mode='complete')
fin = time.time()
nptime = fin-start
print(nptime)

0.0035393238067626953


In [159]:
r2

array([[-10.455804  , -11.123794  , -10.483212  , ..., -10.693069  ,
        -10.598006  , -10.4267845 ],
       [  0.        ,  -3.0491242 ,  -1.6812412 , ...,  -1.5855619 ,
         -1.7610041 ,  -1.9999844 ],
       [  0.        ,   0.        ,  -2.4841995 , ...,  -0.8498958 ,
         -1.0891759 ,  -1.0219033 ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,  -0.3806929 ,
         -0.01374719,   0.47284105],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
         -0.29349664,  -0.41122508],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,  -0.22761676]], dtype=float32)

In [160]:
from pynq import allocate

In [161]:
input_buffer = allocate(2500, np.float32)
out1_buff = allocate(2500, np.float32)
out2_buff = allocate(2500,np.float32)

In [162]:
np.copyto(input_buffer, data_A.flatten())

In [163]:
start_time = time.time()
R_channel.transfer(out2_buff)
A_channel.transfer(input_buffer)
Q_channel.transfer(out1_buff)
R_channel.wait()
A_channel.wait()
Q_channel.wait()
end_time = time.time()
pltime = end_time - start_time
print(pltime)

0.004578828811645508


In [164]:
start = time.time()
q3,r3 = qr_givens(data_A)
fin = time.time()
pstime = fin-start
print(pstime)

0.24099016189575195


In [165]:
pstime/pltime

52.63139807341838

In [167]:
np.linalg.norm(np.abs(r2)- np.abs(r3))

2.4019364e-05

In [168]:
np.linalg.norm(np.abs(q2)- np.abs(q3))

1.3316370646134687e-05

In [135]:
np.sqrt(np.mean((r1 - r3) ** 2))

0.0

In [137]:
np.linalg.norm(r1-r3)

0.0

In [82]:
np.linalg.norm(r1-r3)

0.0

In [138]:
r1

array([[ 1.0455805e+01,  1.1123792e+01,  1.0483209e+01, ...,
         1.0693065e+01,  1.0598007e+01,  1.0426781e+01],
       [ 2.5344441e-08,  3.0491240e+00,  1.6812406e+00, ...,
         1.5855618e+00,  1.7610041e+00,  1.9999850e+00],
       [-6.7215296e-08, -6.0782832e-09,  2.4841998e+00, ...,
         8.4989578e-01,  1.0891758e+00,  1.0219033e+00],
       ...,
       [-5.6869709e-09,  6.8870567e-09,  9.1332284e-09, ...,
         3.8069314e-01,  1.3749514e-02, -4.7284028e-01],
       [-4.9106674e-10, -3.3279708e-09, -1.8858923e-08, ...,
         0.0000000e+00,  2.9349566e-01,  4.1122642e-01],
       [-1.3463685e-07, -3.5253322e-08,  2.0073497e-08, ...,
        -0.0000000e+00, -7.4505806e-09,  2.2761682e-01]], dtype=float32)

10.00000033210277

In [86]:
A = np.random.randn(4, 3)
Q1, R1 = qr_givens(A)
print(np.allclose(Q1 @ R1, A))         
print(np.allclose(Q1.T @ Q1, np.eye(4)))  

True
True


In [87]:
Q1

array([[ 0.1212576 , -0.23572615,  0.64929111, -0.71284699],
       [-0.80051699,  0.0459382 ,  0.5093003 ,  0.31253069],
       [-0.58242548, -0.22955276, -0.56483117, -0.53763541],
       [ 0.07245483, -0.94320114,  0.        ,  0.32422508]])

In [88]:
R1

array([[ 2.62766788e+00,  7.54431531e-01,  1.71512512e+00],
       [-3.88040098e-17,  1.54780321e+00, -2.12279071e-01],
       [ 3.98705411e-17,  0.00000000e+00,  2.03014100e+00],
       [-2.75036119e-17, -5.55111512e-17,  1.39932135e-01]])

In [89]:
Q1 @ R1

array([[ 0.3186247 , -0.27337714,  1.47641399],
       [-2.10349278, -0.53283196, -0.30505399],
       [-1.53042074, -0.79470265, -2.17212271],
       [ 0.19038724, -1.40522754,  0.36986047]])

In [90]:
A

array([[ 0.3186247 , -0.27337714,  1.47641399],
       [-2.10349278, -0.53283196, -0.30505399],
       [-1.53042074, -0.79470265, -2.17212271],
       [ 0.19038724, -1.40522754,  0.36986047]])

In [98]:
Q2, R2 = np.linalg.qr(A, mode='complete')

In [99]:
Q2

array([[-0.1212576 , -0.23572615,  0.59873589, -0.75580759],
       [ 0.80051699,  0.0459382 ,  0.52958567,  0.27676931],
       [ 0.58242548, -0.22955276, -0.60046422, -0.49752267],
       [-0.07245483, -0.94320114,  0.02229506,  0.32345762]])

In [100]:
R2

array([[-2.62766788, -0.75443153, -1.71512512],
       [ 0.        ,  1.54780321, -0.21227907],
       [ 0.        ,  0.        ,  2.03495786],
       [ 0.        ,  0.        ,  0.        ]])

In [101]:
Q1 - Q2

array([[ 2.42515196e-01, -2.77555756e-17,  5.05552205e-02,
         4.29605994e-02],
       [-1.60103398e+00, -1.38777878e-17, -2.02853633e-02,
         3.57613768e-02],
       [-1.16485097e+00, -2.77555756e-17,  3.56330523e-02,
        -4.01127443e-02],
       [ 1.44909666e-01,  2.22044605e-16, -2.22950603e-02,
         7.67458673e-04]])

In [139]:
np.abs(Q1)

array([[0.1212576 , 0.23572615, 0.64929111, 0.71284699],
       [0.80051699, 0.0459382 , 0.5093003 , 0.31253069],
       [0.58242548, 0.22955276, 0.56483117, 0.53763541],
       [0.07245483, 0.94320114, 0.        , 0.32422508]])

In [140]:
np.abs(Q2)

array([[0.1212576 , 0.23572615, 0.59873589, 0.75580759],
       [0.80051699, 0.0459382 , 0.52958567, 0.27676931],
       [0.58242548, 0.22955276, 0.60046422, 0.49752267],
       [0.07245483, 0.94320114, 0.02229506, 0.32345762]])

In [143]:
np.abs(Q1) - np.abs(Q2)

array([[ 8.32667268e-17,  2.77555756e-17,  5.05552205e-02,
        -4.29605994e-02],
       [ 2.22044605e-16, -1.38777878e-17, -2.02853633e-02,
         3.57613768e-02],
       [ 0.00000000e+00,  2.77555756e-17, -3.56330523e-02,
         4.01127443e-02],
       [ 0.00000000e+00, -2.22044605e-16, -2.22950603e-02,
         7.67458673e-04]])

In [175]:
np.min(q3)

-0.40111072544187304

In [176]:
np.min(r3)

-0.8391445

In [177]:
np.max(q3)

0.4363235715965186

In [178]:
np.max(r3)

11.195493